In [14]:
from pathlib import Path
import tomllib
import pandas as pd

with (Path().cwd()/".."/"config.toml").open("rb") as f:
    config = tomllib.load(f)
    config_exp = config[Path().cwd().name]
    config_input = config_exp["input"]

df = pd.read_json(config_exp["output"]["aggregated_table"])
df["savings"] = 100*(1-df["samplns_ub"]/df["baseline_ub"])
df["max_savings"] = 100*(1-df["best_ub"]/df["baseline_ub"])
df

,num_vars,num_clauses,best_ub,best_lb,samplns_ub,samplns_lb,baseline_ub,time_initial_solution,time_last_update_15min,time_last_update_3h,savings,max_savings
calculate,9,15,5,5,5.0,5.0,9,0.349753,0.409805,0.509709,44.444444,44.444444
lcm,9,16,6,6,6.0,6.0,8,0.320516,0.362959,0.459972,25.000000,25.000000
email,10,17,6,6,6.0,6.0,6,0.320861,0.390224,0.476566,0.000000,0.000000
ChatClient,14,20,7,7,7.0,7.0,7,0.370952,1.386273,1.897152,0.000000,0.000000
toybox_2006-10-31_23-30-06,16,13,8,8,8.0,8.0,9,0.392972,1.462752,1.492623,11.111111,11.111111
car,16,33,5,5,5.0,5.0,6,0.285722,0.350269,0.417284,16.666667,16.666667
FeatureIDE,19,27,8,8,8.0,8.0,9,0.372879,270.834257,127.857561,11.111111,11.111111
FameDB,22,40,8,8,8.0,8.0,8,0.325365,1.142090,1.180048,0.000000,0.000000
APL,23,35,7,7,7.0,7.0,9,0.376910,1.216988,1.243059,22.222222,22.222222
SafeBali,24,45,11,11,11.0,11.0,11,0.289868,0.348789,0.371186,0.000000,0.000000


In [29]:
def shorten_name(name):
    threshold = 20
    if len(name) > threshold:
        return (name[: threshold - 3] + "\\ldots").replace("_", "\\_")
    return name.replace("_", "\\_")


def si_num(num):
    return "\\num[text-series-to-math=true]{" + str(int(num)) + "}"


def si_num_bold(num):
    return "\\textbf{" + si_num(num) + "}"

def time_as_latex(time_s):
    secs = round(time_s)
    if secs == 0:
        return "$<\\SI{1}{\\second}$"
    if secs > 900:
        mins = round(secs / 60)
        return "\\SI{" + str(mins) + "}{\\minute}"
    return "\\SI{" + str(secs) + "}{\\second}"

lines = []
for index, row in df.iterrows():
    line = f"{shorten_name(index)} & {si_num(int(row['num_vars']))} & {si_num(int(row['num_clauses']))} "
    # baseline
    if row["best_lb"] == row["baseline_ub"]:
        # bold if optimal
        line += f"& {si_num_bold(row['baseline_ub'])} "
    else:
        line += f"& {si_num(row['baseline_ub'])} "
    # SampLNS UB
    if row["best_lb"] == row["samplns_ub"]:
        # bold if optimal
        line += f"& {si_num_bold(row['samplns_ub'])} "
    else:
        line += f"& {si_num(row['samplns_ub'])} "
    # SampLNS UB 3hr
    if row["best_lb"] == row["best_ub"]:
        # bold if optimal
        line += f" ({si_num_bold(row['best_ub'])}) "
    else:
        line += f" ({si_num(row['best_ub'])}) "
    # SampLNS LB
    if row["samplns_lb"] == row["best_ub"]:
        # bold if optimal
        line += f"& {si_num_bold(row['samplns_lb'])} "
    else:
        line += f"& {si_num(row['samplns_lb'])} "
    # Best LB
    if row["best_lb"] == row["best_ub"]:
        # bold if optimal
        line += f" ({si_num_bold(row['best_lb'])}) "
    else:
        line += f" ({si_num(row['best_lb'])}) "
    # improvement
    line += "& \\SI{"\
        + f"{round(row['savings'], 2):.2g}"\
        + "}{\\percent} (\\SI{"\
        + f"{round(row['max_savings'], 2):.2g}"\
        + "}{\\percent})"
    line += f"& {row['samplns_ub']/row['samplns_lb']:.2f} ({row['best_ub']/row['best_lb']:.2f})"
    line += f"& {time_as_latex(row['time_last_update_15min'])} ({time_as_latex(row['time_last_update_3h'])})"
    line += "\\\\"
    lines.append(line)
    print(line)


calculate & \num[text-series-to-math=true]{9} & \num[text-series-to-math=true]{15} & \num[text-series-to-math=true]{9} & \textbf{\num[text-series-to-math=true]{5}}  (\textbf{\num[text-series-to-math=true]{5}}) & \textbf{\num[text-series-to-math=true]{5}}  (\textbf{\num[text-series-to-math=true]{5}}) & \SI{44}{\percent} (\SI{44}{\percent})& 1.00 (1.00)& $<\SI{1}{\second}$ (\SI{1}{\second})\\
lcm & \num[text-series-to-math=true]{9} & \num[text-series-to-math=true]{16} & \num[text-series-to-math=true]{8} & \textbf{\num[text-series-to-math=true]{6}}  (\textbf{\num[text-series-to-math=true]{6}}) & \textbf{\num[text-series-to-math=true]{6}}  (\textbf{\num[text-series-to-math=true]{6}}) & \SI{25}{\percent} (\SI{25}{\percent})& 1.00 (1.00)& $<\SI{1}{\second}$ ($<\SI{1}{\second}$)\\
email & \num[text-series-to-math=true]{10} & \num[text-series-to-math=true]{17} & \textbf{\num[text-series-to-math=true]{6}} & \textbf{\num[text-series-to-math=true]{6}}  (\textbf{\num[text-series-to-math=true]{6}})

In [37]:
num_instances = len(df)

In [38]:
num_samplns_opt = (df["samplns_lb"] == df["samplns_ub"]).sum()
num_samplns_opt, num_samplns_opt / num_instances

(26, 0.5531914893617021)

In [44]:
num_3h_opt = (df["best_lb"] == df["best_ub"]).sum()
num_3h_opt, num_3h_opt / num_instances

(30, 0.6382978723404256)

In [39]:
num_improvements = (df["savings"] > 0.01).sum()
num_improvements_max = (df["max_savings"] > 0.01).sum()
num_improvements, num_improvements_max, num_improvements / num_instances, num_improvements_max / num_instances

(40, 40, 0.851063829787234, 0.851063829787234)

In [40]:
num_baseline_opt = (df["best_lb"] == df["baseline_ub"]).sum()
num_baseline_opt, num_baseline_opt / num_instances

(7, 0.14893617021276595)

In [47]:

print("\n".join(["\\hline",
 "\\ \\\\",
 "\\textbf{optimality} & & & "+str(num_baseline_opt)+" & $\\geq "+str(num_samplns_opt)+"$ & & \\\\ & & & [\\SI{"+ str(round(100*num_baseline_opt/num_instances))+"}{\\percent}] &[\\SI{"+str(round(100*num_samplns_opt/num_instances))+"}{\\percent}] & & \\\\ ",
 "\\textbf{improvements} & & & & &\\multicolumn{2}{r}{\\num[text-series-to-math=true]{"+str(num_improvements)+"} [\\SI{"+str(round(100*num_improvements/num_instances))+"}{\\percent}]} \\\\ \\ \\\\ \\ \\\\",
"",
"\\end{tabular}",
])
)

\hline
\ \\
\textbf{optimality} & & & 7 & $\geq 26$ & & \\ & & & [\SI{15}{\percent}] &[\SI{55}{\percent}] & & \\ 
\textbf{improvements} & & & & &\multicolumn{2}{r}{\num[text-series-to-math=true]{40} [\SI{85}{\percent}]} \\ \ \\ \ \\

\end{tabular}


In [51]:
# number of baseline instances at least worse than 25% of the optimum
num_baseline_25 = (df["savings"]>=20).sum()
num_baseline_25, num_baseline_25 / num_instances

(28, 0.5957446808510638)

In [54]:
num_baseline_10 = (df["savings"]>=10).sum()
num_baseline_10, num_baseline_10 / num_instances

(33, 0.7021276595744681)

In [55]:
df["savings"].mean()

22.392428166143418

In [60]:
(df["samplns_ub"]/df["best_lb"]<=1.2).sum()/num_instances

0.8297872340425532